In [1]:
import os
import django

os.environ.setdefault(
    'DJANGO_SETTINGS_MODULE',
    'norton.settings.production',
)

django.setup()

# Authors, ranked by total number of works and excerpts

In [26]:
from django.db.models import Count, Case, When

from data.models import Author

In [59]:
work_counts = Author.objects.raw('''
    SELECT data_author.*, COUNT(data_work.id) AS num_works
    FROM data_author
    JOIN data_work ON data_work.author_id = data_author.id
    GROUP BY data_author.id
    ORDER BY num_works DESC
''')

for author in work_counts:
    print(author.num_works, author)

95 Unknown
87 William Wordsworth
81 Emily Dickinson
56 Sir Philip Sidney
54 John Donne
54 William Blake
50 William Shakespeare
46 Percy Bysshe Shelley
45 William Butler Yeats
44 Walt Whitman
42 Samuel Taylor Coleridge
38 John Keats
38 Edmund Spenser
36 George Gordon, Lord Byron
34 Ben Jonson
29 Samuel Johnson
29 William Bradford
27 Herman Melville
27 Alfred, Lord Tennyson
25 George Herbert
25 Matthew Arnold
25 Geoffrey Chaucer
25 Robert Browning
25 Robert Herrick
22 Thomas Hardy
22 Robert Frost
21 Roger Williams
21 John Dryden
20 Edgar Allan Poe
20 Olaudah Equiano
20 Ralph Waldo Emerson
20 Christina Rossetti
20 John Milton
20 Sir Thomas Wyatt the Elder
19 Anne Bradstreet
18 Gerard Manley Hopkins
18 Elizabeth I
18 Edward Taylor
18 Thomas Carlyle
18 Mary Wollstonecraft
18 Langston Hughes
18 Mary Wroth
18 Elizabeth Barrett Browning
17 William Carlos Williams
17 Zitkala Sâ (Gertrude Simmons Bonnin)
17 Jonathan Swift
17 Sir Francis Bacon
17 Harriet Beecher Stowe
16 Theodore Roethke
16 Willi

# Authors, ranked by total number of excerpts

In [87]:
work_counts = Author.objects.raw('''
    SELECT data_author.*, COUNT(work.id) AS num_works
    FROM data_author
    JOIN data_work AS work ON work.author_id = data_author.id
    LEFT OUTER JOIN data_work as child on child.parent_id = work.id
    WHERE child.id IS NULL
    GROUP BY data_author.id
    ORDER BY num_works DESC
''')

data = [
    (
        author.num_works,
        author.name,
        author.gender.name if author.gender else None,
        ', '.join([r.name for r in author.race.all()]),
        ', '.join([g.name for g in author.genre.all()]),
    )
    for author in work_counts
]

In [88]:
import pandas as pd

df = pd.DataFrame(data, columns=('works', 'name', 'gender', 'race', 'genre'))

pd.set_option('display.max_rows', 1000)

df.head(500)

,works,name,gender,race,genre
0,80,Emily Dickinson,Female,White or European American,Poetry
1,75,Unknown,None,,
2,68,William Wordsworth,Male,White or European American,Poetry
3,53,Sir Philip Sidney,Male,White or European American,"Poetry, Essay / Criticism"
4,47,John Donne,Male,White or European American,Poetry
5,46,William Blake,Male,White or European American,Poetry
6,46,William Shakespeare,Male,White or European American,"Poetry, Drama"
7,42,William Butler Yeats,Male,White or European American,"Poetry, Personal Essay / Memoir"
8,37,Percy Bysshe Shelley,Male,White or European American,"Poetry, Drama, Essay / Criticism"
9,35,Walt Whitman,Male,White or European American,"Poetry, Personal Essay / Memoir"


# Racial categories, ranked by total number of works

In [57]:
race_counts = Author.objects.raw('''
    SELECT data_race.*, COUNT(data_work.id) AS num_works
    FROM data_race
    JOIN data_author_race on data_author_race.race_id = data_race.id
    JOIN data_author on data_author.id = data_author_race.author_id
    JOIN data_work on data_work.author_id = data_author.id
    GROUP BY data_race.id
    ORDER BY num_works DESC
''')

for race in race_counts:
    print(race.num_works, race)

3304 White or European American
260 Black or African American
108 Native American, American Indian, or Alaska Native
64 Hispanic or Latino
55 Jewish
21 East Asian
16 South Asian
3 Middle Eastern
1 African
1 Native Hawaiian or other Pacific Islander
